# Tool Registration and Acrion Registration Explained

First we build a tool registry with decorator. This helps register all the tools in the same format and be able to use them by the agent

In [1]:
# Tool registration
tools = {}

def register_tool(tags=None):
    def decorator(func):
        tools[func.__name__] = {
            "function": func,
            "description": func.__doc__ or "No description",
            "tags": tags or []
        }
        return func
    return decorator

Here are some examples of the tools that get registered in our tool registry when we use @register_tool

In [8]:
# Step 2: Register some simple tools (like your file_tools.py)
@register_tool(tags=["math"])
def add_numbers(a: int, b: int) -> int:
    """Adds two numbers together"""
    return a + b

@register_tool(tags=["text"])
def make_uppercase(text: str) -> str:
    """Converts text to uppercase"""
    return text.upper()

@register_tool(tags=["system"])
def say_goodbye(message: str = "Goodbye!") -> str:
    """Says goodbye"""
    return f"Agent says: {message}"

Lets see the list of tools available

In [9]:
tools.keys()

dict_keys(['add_numbers', 'make_uppercase', 'say_goodbye'])

We define an action class that takes the name, function and description of a tool and enables exceution

In [10]:
# Step 3: Create Action class (like your GAME.py)
class Action:
    def __init__(self, name, function, description):
        self.name = name
        self.function = function
        self.description = description
    
    def execute(self, **args):
        return self.function(**args)

In the action registry, which is the second layer of registry for tools, we can register actions and filter available actions. For example, all the tools that have a specific tag get into the same group, and then later when we look for available actions we can:

Filter by relevance: Only show tools that are relevant to the current agent's purpose (e.g., only "file_operations" and "system" tools for a file management agent)
Organize by capability: Group related functions together (e.g., all "math" tools, all "text" tools, all "database" tools)
Control access: Limit which tools different parts of the system can see and execute
Provide a clean interface: Convert raw tool dictionary entries into standardized Action objects that are easier to work with
Enable dynamic selection: At runtime, the agent can request only the tools it needs for a specific task, rather than being overwhelmed with all available tools

Example: An agent focused on file operations doesn't need to see text processing or mathematical tools - the ActionRegistry filters out everything except tools tagged with "file_operations" and "system".

In [11]:
# Step 4: Create ActionRegistry that filters tools 
class SimpleActionRegistry:
    def __init__(self, tags=None):
        self.actions = {}
        
        # Filter tools from the global registry by tags
        for tool_name, tool_info in tools.items():
            tool_tags = tool_info.get("tags", [])
            
            # Only include tools that match our desired tags
            if tags and not any(tag in tool_tags for tag in tags):
                continue
                
            # Convert tool to Action object
            action = Action(
                name=tool_name,
                function=tool_info["function"],
                description=tool_info["description"]
            )
            self.actions[tool_name] = action
    
    def get_available_actions(self):
        return list(self.actions.keys())
    
    def execute_action(self, action_name, **args):
        if action_name in self.actions:
            return self.actions[action_name].execute(**args)
        else:
            return f"Action '{action_name}' not found"

In [12]:
# Step 5: Test the two-layer system
if __name__ == "__main__":
    print("=== LAYER 1: Raw Tools Dictionary ===")
    print("All registered tools:", list(tools.keys()))
    for name, info in tools.items():
        print(f"- {name}: tags={info['tags']}")
    
    print("\n=== LAYER 2: Filtered Action Registry ===")
    
    # Create registry that only wants math tools
    math_registry = SimpleActionRegistry(tags=["math"])
    print("Math registry has:", math_registry.get_available_actions())
    
    # Create registry that only wants text tools  
    text_registry = SimpleActionRegistry(tags=["text"])
    print("Text registry has:", text_registry.get_available_actions())
    
    # Create registry that wants math AND text tools
    multi_registry = SimpleActionRegistry(tags=["math", "text"])
    print("Multi registry has:", multi_registry.get_available_actions())
    
    print("\n=== EXECUTION EXAMPLES ===")
    result1 = math_registry.execute_action("add_numbers", a=5, b=3)
    print(f"Math registry: add_numbers(5, 3) = {result1}")
    
    result2 = text_registry.execute_action("make_uppercase", text="hello world")
    print(f"Text registry: make_uppercase('hello world') = {result2}")
    
    # Try to use wrong tool in wrong registry
    result3 = math_registry.execute_action("make_uppercase", text="hello")
    print(f"Math registry trying text tool: {result3}")

=== LAYER 1: Raw Tools Dictionary ===
All registered tools: ['add_numbers', 'make_uppercase', 'say_goodbye']
- add_numbers: tags=['math']
- make_uppercase: tags=['text']
- say_goodbye: tags=['system']

=== LAYER 2: Filtered Action Registry ===
Math registry has: ['add_numbers']
Text registry has: ['make_uppercase']
Multi registry has: ['add_numbers', 'make_uppercase']

=== EXECUTION EXAMPLES ===
Math registry: add_numbers(5, 3) = 8
Text registry: make_uppercase('hello world') = HELLO WORLD
Math registry trying text tool: Action 'make_uppercase' not found
